<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_VAQ_VQE(MolecularGeometry_H2_MinimizedCode)_mkbahk_20251208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python 3.9 ~ 3.11 환경에서 아래 순서대로 설치
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install qiskit-algorithms==0.3.1
%pip install qiskit-nature[pyscf]==0.7.2
#%pip install pyscf==2.6.0
%pip install py3Dmol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 15.7 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=d98330c0960dff8d30c62d940125e314a4ab086a11c6cfec4ef48d40f783a00b
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import datetime
import time

# 분자 구조(좌표)를 입력할 때 길이의 단위를 명확하게 지정하기 위한 열거형(Enum) 클래스, Ångström이야? Bohr이야? nm이야?”
from qiskit_nature.units import DistanceUnit

#First quantization(제1 양자화): 실제 분자의 전자 구조(오비탈, 에너지, 1체/2체 적분 등)를 고전적으로 정확히 계산해서 양자 컴퓨터가 쓸 수 있는 2차 양자화 해밀토니안으로 바꿔주는 핵심 브릿지 역할, 쉽게 말해,
# H₂, H₂O, CH₄ 같은 분자를 주면, PySCF(하트리-포크 또는 DFT)로 분자 오비탈과 모든 적분을 계산해서 Qiskit이 VQE나 QAOA에 바로 넣을 수 있는 Pauli 문자열 해밀토니안으로 변환해 줌”
from qiskit_nature.second_q.drivers import PySCFDriver, Psi4Driver, GaussianDriver, ElectronicStructureDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo

#Second quantization(제2 양자화): 양자 다체계 문제를 효율적으로 모델링하기 위한 핵심 기법입니다. 이는 생성(creation)과 소멸(annihilation) 연산자를 사용해 해밀토니안을 표현하는 방식으로,
# 각 Problem 클래스(ElectronicStructureProblem, VibrationalStructureProblem, LatticeModelProblem)는 second_q_ops() 메서드를 통해
# second-quantized operator(주로 FermionicOp 또는 BosonicOp 형태)를 생성합니다. 이 연산자는 이후 QubitMapper(Jordan-Wigner, Parity 등)를 통해 큐비트 연산자로 변환되어 양자 알고리즘(VQE 등)에서 사용
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, BravyiKitaevMapper, TaperedQubitMapper, DirectMapper, QubitMapper

#실행하기 전에 분자 해밀토니안을 작고 계산 가능한 크기로 줄여주는 전처리 도구
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer, BasisTransformer

#from qiskit_nature.second_q.mappers.QubitConverter import QubitConverter
from qiskit_nature.second_q.problems import ElectronicStructureProblem, VibrationalStructureProblem, LatticeModelProblem
from qiskit_nature.second_q.algorithms import GroundStateSolver, GroundStateEigensolver, ExcitedStatesSolver, ExcitedStatesEigensolver, QEOM, QEOMResult, EvaluationRule

#Classical Optimizer
from qiskit_algorithms.optimizers import SLSQP, COBYLA, L_BFGS_B, SPSA, NFT, P_BFGS, QNSPSA, P_BFGS, ADAM, adam_amsgrad, NELDER_MEAD, POWELL, TNC, CG, NFT, IMFIL, SNOBFIT, BOBYQA, GSLS, AQGD

#Aanstz
from qiskit.circuit.library import TwoLocal, EfficientSU2, RealAmplitudes, ExcitationPreserving, PauliTwoDesign, NLocal

#Hartree-Fock (HF) 방법은 양자 화학에서 다전자 원자나 분자의 전자 구조를 근사적으로 계산하는 기본적인 방법입니다.
#"Hartree-Fock solution", 이 방법으로 얻어진 최적화된 파동함수와 에너지를 의미하며, 정확한 슈뢰딩거 방정식 해가 아닌 최선의 단일 Slater determinant 근사
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

#from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator

from qiskit_algorithms import VQE

In [37]:
# 1. 지오메트리 설정 (실험적 최적 거리 0.735 Angstrom)
molecule = "H 0.0 0.0 0.0; H 0.0 0.0 0.735"

In [38]:
driver = PySCFDriver(
    atom=molecule,
    #basis="cc-pvdz",       # sto-3g보다 훨씬 정밀한 기저 집합
    basis="sto-3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

In [39]:
# 2. 양자 문제 생성
problem = driver.run()
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.second_q_ops()[0])

In [40]:
print(qubit_op)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.12091263+0.j,
  0.17218393+0.j,  0.16892754+0.j, -0.22575349+0.j,  0.16614543+0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])


In [41]:
# 3. Ansatz 구성 (UCCSD)
init_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
)

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=init_state,
)

In [42]:
# 4. 고정밀 Optimizer 설정 (L_BFGS_B)
# tolerance를 낮게 잡아 더 정밀하게 수렴하도록 유도합니다.
optimizer = L_BFGS_B(maxiter=1000, ftol=1e-12)

In [45]:
# 5. VQE 실행 with Callback

intermediate_values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    intermediate_values.append({
        "eval_count": eval_count,
        "parameters": np.array(parameters).tolist(), # Modified this line
        "mean": mean,
        "std": std,
        "time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    print(f"Iteration {eval_count}: Energy = {mean:.8f}")
###def

# 기존 VQE solver를 재사용하되 callback 함수를 추가합니다.
vqe_solver_with_callback = VQE(Estimator(), ansatz, optimizer, callback=store_intermediate_result)
vqe_solver_with_callback.initial_point = np.zeros(ansatz.num_parameters)

print("Running VQE with callback...")
result_with_callback = vqe_solver_with_callback.compute_minimum_eigenvalue(operator=qubit_op)
print("VQE with callback finished.")

Running VQE with callback...
Iteration 1: Energy = -1.83696799
Iteration 2: Energy = -1.83696799
Iteration 3: Energy = -1.83696799
Iteration 4: Energy = -1.83696799
Iteration 5: Energy = -0.87441261
Iteration 6: Energy = -0.87441261
Iteration 7: Energy = -0.87441261
Iteration 8: Energy = -0.87441263
Iteration 9: Energy = -1.85678494
Iteration 10: Energy = -1.85678494
Iteration 11: Energy = -1.85678494
Iteration 12: Energy = -1.85678494
Iteration 13: Energy = -1.85727498
Iteration 14: Energy = -1.85727498
Iteration 15: Energy = -1.85727498
Iteration 16: Energy = -1.85727498
Iteration 17: Energy = -1.85727503
Iteration 18: Energy = -1.85727503
Iteration 19: Energy = -1.85727503
Iteration 20: Energy = -1.85727503
VQE with callback finished.


In [47]:
# 6. 결과 해석
interpretation = problem.interpret(result_with_callback)
total_energy = interpretation.total_energies[0]

print(f"Calculated Total Energy: {total_energy:.8f} Hartree")
print(f"Target Energy: -1.17447 Hartree")
print(f"Error: {abs(total_energy - (-1.17447)):.8f} Hartree")

Calculated Total Energy: -1.13730604 Hartree
Target Energy: -1.17447 Hartree
Error: 0.03716396 Hartree
